In [1]:
with open('inputs/07.txt') as file:
    input = file.read()

from itertools import permutations


In [2]:
def run(mem):
    i = 0
    def get(j,m):
        p = i+j
        if m == 0:
            p = mem[p]
        return p
    while 1:
        s = ('0000'+str(mem[i]))[-5:]
        a,b,c,op = int(s[0]), int(s[1]), int(s[2]), int(s[3:])
        if op == 1:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            mem[p] = mem[x] + mem[y]
            i += 4
        elif op == 2:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            mem[p] = mem[x] * mem[y]
            i += 4
        elif op == 3:
            p = get(1,c)
            mem[p] = yield
            i += 2
        elif op == 4:
            p = get(1,c)
            yield mem[p]
            i += 2
        elif op == 5:
            x = get(1,c)
            y = get(2,b)
            if mem[x]:
                i = mem[y]
            else:
                i += 3
        elif op == 6:
            x = get(1,c)
            y = get(2,b)
            if not mem[x]:
                i = mem[y]
            else:
                i += 3
        elif op == 7:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            mem[p] = int(mem[x] < mem[y])
            i += 4
        elif op == 8:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            mem[p] = int(mem[x] == mem[y])
            i += 4
        elif op == 99:
            return
        else:
            raise Exception('Unknown opcode', op)
            break


In [3]:
%%time

prog = list(map(int, input.split(',')))

maxsignal = 0
for amps in permutations(range(5),5):
    signal = 0
    for amp in amps:
        proc = run(prog[:])
        next(proc)
        proc.send(amp)
        signal = proc.send(signal)
    if maxsignal<signal: maxsignal = signal
maxsignal


Wall time: 22.9 ms


95757

In [4]:
%%time

# prog = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
# prog = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
prog = list(map(int, input.split(',')))


maxsignal = 0
for amps in permutations(range(5,10),5):
    procs = []
    for amp in amps:
        proc = run(prog[:])
        next(proc)
        proc.send(amp)
        procs.append(proc)
    signal = 0
    try:
        for proc in procs:
            signal = proc.send(signal)
        while 1:
            for proc in procs:
                next(proc) # last was output yield, so we need to wait for input yield.
                signal = proc.send(signal)
    except:
        if maxsignal<signal: maxsignal = signal

maxsignal


Wall time: 78.8 ms


4275738